In [2]:
# main.py (excerpt)

import os, sys, json

sys.path.append("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt")

import gymnasium as gym
import gymnasium_robotics
import mujoco
import numpy as np
# ignore logging warninngs
import logging
logging.getLogger('mujoco').setLevel(logging.ERROR)

from utils.change_shape import get_geom_info, apply_override

os.environ['MUJOCO_GL'] = 'egl'
os.environ['PYOPENGL_PLATFORM'] = 'egl'

config = json.load(open("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt/configs/test.json"))

gym.register_envs(gymnasium_robotics)
env = gym.make(config["env_name"], render_mode="rgb_array")
env.reset()

# Log initial object geom info (by name, not index)
obj_info_init = get_geom_info(env, "object0")
config["object_type_initial"] = obj_info_init["type"]
config["object_size_initial"] = obj_info_init["size"].tolist()
config["object_gid"] = obj_info_init["gid"]

# "override_object": {"name":"object0","type":"ellipsoid","size":[0.025,0.02,0.015],"mass":2.0,"update_inertia":true}
ovr = config.get("override_object")
if ovr:
    result = apply_override(
        env,
        name=ovr.get("name", "object0"),
        typ=ovr["type"],
        size=ovr["size"],
        mass=ovr.get("mass", None),
        update_inertia=bool(ovr.get("update_inertia", False)),
        atol=1e-9,
    )
    print("Geom override:", result["ok"], result["after"])
    config["object_type_final"] = result["after"]["type"]
    config["object_size_final"] = np.asarray(result["after"]["size"]).tolist()
    config["object_inertia_updated"] = result["inertia_updated"]


Geom override: True {'gid': 23, 'type': 'ellipsoid', 'size': array([0.025, 0.02 , 0.015]), 'dataid': -1, 'body_id': 32}


In [ ]:
import os, sys, json
sys.path.append("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt")

import gymnasium as gym, gymnasium_robotics, mujoco, numpy as np
from utils.change_shape import get_geom_info, apply_override, snapshot_geom_and_body, jprint

os.environ['MUJOCO_GL'] = 'egl'
os.environ['PYOPENGL_PLATFORM'] = 'egl'

config = json.load(open("/home/ul/ul_student/ul_cep22/my_folders/drl_projekt/configs/test.json"))

gym.register_envs(gymnasium_robotics)
env = gym.make(config["env_name"], render_mode="rgb_array")
env.reset()

snap_before = snapshot_geom_and_body(env, "object0")
jprint("[BEFORE  XML/COMPILED]", snap_before)

config["object_gid"]            = snap_before["geom"]["gid"]
config["object_type_initial"]   = snap_before["geom"]["type"]
config["object_size_initial"]   = snap_before["geom"]["size"]
config["body_mass_initial"]     = snap_before["body"]["mass"]
config["body_inertia_initial"]  = snap_before["body"]["inertia_diag"]

ovr = config.get("override_object")
if ovr:
    jprint("[REQUESTED OVERRIDE]", {
        "name": ovr.get("name", "object0"),
        "type": ovr.get("type"),
        "size": ovr.get("size"),
        "mass": ovr.get("mass", None),
        "update_inertia": bool(ovr.get("update_inertia", False)),
    })

    # Apply override
    result = apply_override(
        env,
        name=ovr.get("name", "object0"),
        typ=ovr["type"],
        size=ovr["size"],
        mass=ovr.get("mass", None),
        update_inertia=bool(ovr.get("update_inertia", False)),
        atol=1e-9,
    )

    snap_after = snapshot_geom_and_body(env, ovr.get("name", "object0"))
    jprint("[AFTER  LIVE MODEL]", snap_after)

    print("Geom override OK?", result["ok"])

    config["object_type_final"]      = snap_after["geom"]["type"]
    config["object_size_final"]      = snap_after["geom"]["size"]
    config["body_mass_final"]        = snap_after["body"]["mass"]
    config["body_inertia_final"]     = snap_after["body"]["inertia_diag"]
    config["object_inertia_updated"] = result["inertia_updated"]


[BEFORE  XML/COMPILED] {
  "geom": {
    "name": "object0",
    "gid": 23,
    "type": "box",
    "size": [
      0.025,
      0.025,
      0.025
    ],
    "dataid": -1
  },
  "body": {
    "bid": 32,
    "mass": 2.0,
    "inertia_diag": [
      0.0008333333333333335,
      0.0008333333333333335,
      0.0008333333333333335
    ]
  }
}
[REQUESTED OVERRIDE] {
  "name": "object0",
  "type": "ellipsoid",
  "size": [
    0.025,
    0.02,
    0.015
  ],
  "mass": 2.0,
  "update_inertia": true
}
[AFTER  LIVE MODEL] {
  "geom": {
    "name": "object0",
    "gid": 23,
    "type": "ellipsoid",
    "size": [
      0.025,
      0.02,
      0.015
    ],
    "dataid": -1
  },
  "body": {
    "bid": 32,
    "mass": 2.0,
    "inertia_diag": [
      0.00025,
      0.00034,
      0.00041000000000000005
    ]
  }
}
Geom override OK? True
